Import packages

In [49]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Linear

import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import SAGEConv, to_hetero

Define which device will be used

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Getting the MovieLens dataset

In [51]:
%%capture
path = osp.abspath('')
dataset = MovieLens(path, model_name='all-MiniLM-L6-v2')

In [52]:
data = dataset[0].to(device)
data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836]
  }
)

Add user node features for message passing

In [53]:
data['user'].x = torch.eye(data['user'].num_nodes, device=device)
del data['user'].num_nodes
data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836]
  }
)

Add a reversed relations for message passing

In [54]:
data = T.ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label
data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836]
  },
  (movie, rev_rates, user)={ edge_index=[2, 100836] }
)

Split dataset in link level into train, validate and test sets

In [55]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)(data)

In [56]:
train_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 80670],
    edge_label=[80670],
    edge_label_index=[2, 80670]
  },
  (movie, rev_rates, user)={ edge_index=[2, 80670] }
)

In [57]:
val_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 80670],
    edge_label=[10083],
    edge_label_index=[2, 10083]
  },
  (movie, rev_rates, user)={ edge_index=[2, 80670] }
)

In [58]:
test_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 90753],
    edge_label=[10083],
    edge_label_index=[2, 10083]
  },
  (movie, rev_rates, user)={ edge_index=[2, 90753] }
)

Defining the weighted loss

In [59]:
weight = torch.bincount(train_data['user', 'movie'].edge_label)
weight = weight.max() / weight

def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

weight

tensor([25.7292,  7.6899,  2.6580,  1.0576,  1.0000,  2.6460], device='cuda:0')

Encoder

In [60]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

Decoder

In [61]:

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

Model

In [62]:

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

The main model creation

In [63]:
model = Model(hidden_channels=32).to(device)

Passing the data in model

In [64]:
with torch.no_grad():
    model.encoder(train_data.x_dict, train_data.edge_index_dict)

Defining the optimizer

In [65]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Train iteration

In [66]:
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'movie'].edge_label_index)
    target = train_data['user', 'movie'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)

Test iteration

In [67]:
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

Main processing cycle

In [69]:
for epoch in range(1, 301):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
            f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Epoch: 010, Loss: 2.4379, Train: 1.0074, Val: 1.0925, Test: 1.1028
Epoch: 020, Loss: 2.4153, Train: 1.0184, Val: 1.1039, Test: 1.1172
Epoch: 030, Loss: 2.3748, Train: 1.0409, Val: 1.1271, Test: 1.1423
Epoch: 040, Loss: 2.3514, Train: 1.0096, Val: 1.1005, Test: 1.1136
Epoch: 050, Loss: 2.3208, Train: 1.0193, Val: 1.1116, Test: 1.1266
Epoch: 060, Loss: 2.4110, Train: 0.9907, Val: 1.0853, Test: 1.0959
Epoch: 070, Loss: 2.3654, Train: 1.0098, Val: 1.1062, Test: 1.1213
Epoch: 080, Loss: 2.2800, Train: 1.0237, Val: 1.1169, Test: 1.1329
Epoch: 090, Loss: 2.2681, Train: 1.0005, Val: 1.0999, Test: 1.1154
Epoch: 100, Loss: 2.2471, Train: 1.0051, Val: 1.1056, Test: 1.1224
Epoch: 110, Loss: 2.2290, Train: 1.0026, Val: 1.1052, Test: 1.1235
Epoch: 120, Loss: 2.2189, Train: 0.9978, Val: 1.1037, Test: 1.1218
Epoch: 130, Loss: 2.2220, Train: 1.0160, Val: 1.1206, Test: 1.1406
Epoch: 140, Loss: 2.2242, Train: 0.9848, Val: 1.0912, Test: 1.1089
Epoch: 150, Loss: 2.2085, Train: 1.0048, Val: 1.1139, Test: 1.